In [17]:
import collections
import torch
import math 
from torch import nn
import plotly.graph_objects as go
from d2l.torch import d2l

生成数据

In [32]:
n_train = 50
x_train, _ = torch.sort(torch.randn(n_train)  * 5)

def f(x):
    return 2 * torch.sin(x) + x**0.8

y_train = f(x_train) + torch.normal(0.0, 0.5, (n_train,))
print(y_train)

x_test = torch.arange(0,5,0.1)
y_truth = f(x_test)

n_test = len(x_test)
n_test

tensor([        nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,  2.3615e-01,  9.3529e-01, -8.7742e-03,
         1.8095e+00,  1.1144e+00,  1.9450e+00,  2.0524e+00,  3.0373e+00,
         3.8463e+00,  2.5025e+00,  3.0116e+00,  2.0100e+00,  1.8790e+00,
         1.8494e+00,  1.5174e+00,  1.3038e+00,  1.7053e+00,  1.2564e+00,
         7.7077e-01,  1.4971e+00,  1.6905e+00,  4.0829e+00,  6.4168e+00,
         6.1690e+00,  5.7416e+00,  3.6803e+00,  7.8002e+00,  9.2027e+00])


50

可视化

In [23]:
y_hat = torch.repeat_interleave(y_train.mean(), n_test)
print(y_hat)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_train, y=y_train,mode='markers'))
fig.add_trace(go.Scatter(x=x_test, y=y_truth))
fig.show()

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan])
